# Puzzle 1: Calculate the highest possible amp signal

### Unique phase setting between 0 and 4 for any amp --> what is the highest signal?

In [1]:
import numpy as np
from copy import deepcopy

## Load input

In [2]:
with open('./input7.txt', 'r') as file:

    software = file.readlines()

#### Convert data to list of integers

In [3]:
software = list(map(int, software[0].split(',')))

## Calculation

In [60]:
def intcode_computer(inputs, data):
    i = 0
    run = True

    while run is True:
        if len(str(data[i])) < 5:
            add = (5-len(str(data[i])))*'0'
            data[i] = '{0}{1}'.format(add, str(data[i]))

        optcode = data[i][-2:]
        mode1 = data[i][-3]
        mode2 = data[i][-4]
        mode3 = data[i][-5]

        if mode1 == '0' and optcode != '99':
            param1 = int(data[data[i+1]])
        else:
            param1 = int(data[i+1])

        if optcode == '01':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])
            data[data[i+3]] = param1 + param2
            i += 4

        if optcode == '02':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])
            data[data[i+3]] = param1 * param2
            i += 4

        if optcode == '03':
            data[data[i+1]] = inputs[0]
            del inputs[0]
            i += 2

        if optcode == '04':
            if mode1 == '0':
                out = data[data[i+1]]
                # print(data[data[i+1]])
            else:
                out = data[data[i+1]]
                # print(data[i+1])
            i += 2

        if optcode == '05':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])

            if param1 != 0:
                i = param2
            else:
                i += 3

        if optcode == '06':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])

            if param1 == 0:
                i = param2
            else:
                i += 3

        if optcode == '07':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])

            if param1 < param2:
                data[data[i+3]] = 1
            else:
                data[data[i+3]] = 0
            i += 4

        if optcode == '08':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])

            if param1 == param2:
                data[data[i+3]] = 1
            else:
                data[data[i+3]] = 0
            i += 4

        if optcode == '99':
            run = False
    return out

In [34]:
best_output = 0

for pA in range(0, 5):
    outA = intcode_computer([pA, 0], deepcopy(software))
    for pB in range(0, 5):
        if pA == pB:
            outB = False
        else:
            outB = intcode_computer([pB, outA], deepcopy(software))
        for pC in range(0, 5):
            if pC in [pA, pB] or outB == False:
                outC = False
            else:
                outC = intcode_computer([pC, outB], deepcopy(software))
            for pD in range(0, 5):
                if pD in [pA, pB, pC] or outC == False or outB == False:
                    outD = False
                else:
                    outD = intcode_computer([pD, outC], deepcopy(software))
                for pE in range(0, 5):
                    if pE in [pA, pB, pC, pD] or outD == False or outC == False or outB == False:
                        outE = 0
                    else:
                        outE = intcode_computer([pE, outD], deepcopy(software))
                    
                    if outE > best_output:
                        best_output = outE
                        best_combination = [pA, pB, pC, pD, pE]

In [35]:
print('The highest signal is {0}.'.format(best_output))

The highest signal is 77500.


# Puzzle 2:  Calculate the highest possible amp signal using a feedback loop

### phase settings from 5 to 9

In [25]:
class intcode_computer(object):
    
    def __init__(self, phase, data):
        
        self.phase = phase
        self.data = data
        self.inputs = [phase]
        self.i = 0

    def step(self, amp_in):
        self.inputs.append(amp_in)
        self.run = True
        self.done = False
        self.out = 0

        while self.run is True:
            if len(str(self.data[self.i])) < 5:
                add = (5-len(str(self.data[self.i])))*'0'
                self.data[self.i] = '{0}{1}'.format(add, str(self.data[self.i]))

            optcode = self.data[self.i][-2:]
            mode1 = self.data[self.i][-3]
            mode2 = self.data[self.i][-4]
            mode3 = self.data[self.i][-5]
            print(optcode)

            if mode1 == '0' and optcode != '99':
                param1 = int(self.data[self.data[self.i+1]])
            else:
                if optcode != '99':
                    param1 = int(self.data[self.i+1])

            if optcode == '01':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])
                self.data[self.data[self.i+3]] = param1 + param2
                self.i += 4

            if optcode == '02':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])
                self.data[self.data[self.i+3]] = param1 * param2
                self.i += 4

            if optcode == '03':
                if len(self.inputs) > 0:
                    self.data[self.data[self.i+1]] = self.inputs[0]
                    del self.inputs[0]
                    self.i += 2
                else:
                    self.run = False
                    print('Amp waits for input')
                    break
                    
            if optcode == '04':
                if mode1 == '0':
                    self.out = self.data[self.data[self.i+1]]
                    #print(self.data[self.data[self.i+1]])
                else:
                    self.out = self.data[self.data[self.i+1]]
                    #print(self.data[self.i+1])
                self.i += 2

            if optcode == '05':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])

                if param1 != 0:
                    self.i = param2
                else:
                    self.i += 3

            if optcode == '06':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])

                if param1 == 0:
                    self.i = param2
                else:
                    self.i += 3

            if optcode == '07':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])

                if param1 < param2:
                    self.data[self.data[self.i+3]] = 1
                else:
                    self.data[self.data[self.i+3]] = 0
                self.i += 4

            if optcode == '08':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])

                if param1 == param2:
                    self.data[self.data[self.i+3]] = 1
                else:
                    self.data[self.data[self.i+3]] = 0
                self.i += 4

            if optcode == '99':
                self.run = False
                self.i += 1
                self.done = True
        return self.out, self.done

#### generate all possible phase settings

In [26]:
from itertools import permutations 

In [27]:
phase_settings = list(permutations(range(5, 10))) 

In [28]:
best_output = 0

for phase in phase_settings:
    done = [False]*5
    outE = 0
    
    ampA = intcode_computer(phase[0], deepcopy(software))
    ampB = intcode_computer(phase[1], deepcopy(software))
    ampC = intcode_computer(phase[2], deepcopy(software))
    ampD = intcode_computer(phase[3], deepcopy(software))
    ampE = intcode_computer(phase[4], deepcopy(software))
    
    print(phase)
    while True not in done:
        outA, done[0] = ampA.step(outE)
        outB, done[1] = ampB.step(outA)
        outC, done[2] = ampC.step(outB)
        outD, done[3] = ampD.step(outC)
        outE, done[4] = ampE.step(outD)
        if True in done and outE > best_output:
            best_output = outE

print('The value of the highest possible amp signal is {0}.',format(best_output))

(5, 6, 7, 8, 9)
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
02
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03

Amp waits for input
03
01
04
99
03
02
04
99
03
01
04
99
03
01
04
99
03
02
04
99
(5, 7, 9, 6, 8)
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
02
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for i

04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
99
03
01
04
99
03
01
04
99
03
02
04
99
03
02
04
99
(5, 9, 6, 7, 8)
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
02
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits

(6, 5, 8, 7, 9)
03
01
05
03
02
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03

03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
99
03
02
04
99
03
01
04
99
03
01
04
99
03
01
04
99
(6, 7, 9, 8, 5)
03
01
05
03
02
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits fo

...

03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
99
03
02
04
99
03
01
04
99
03
02
04
99
03
01
04
99
(8, 7, 5, 9, 6)
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp

03
01
05
03
02
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
A

(9, 5, 8, 6, 7)
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
02
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03

03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
99
03
02
04
99
03
01
04
99
03
02
04
99
03
01
04
99
(9, 7, 5, 6, 8)
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
02
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp

02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
99
03
01
04
99
03
02
04
99
03
01
04
99
03
02
04
99
(9, 8, 6, 7, 5)
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
02
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
05
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
02
04
03
Amp waits for input
03
01
04
03
Amp waits for input
03
01
04
03
Amp wa

In [24]:
ampE.data[510]

'00099'

In [13]:
done = [True,True]
while True in done:
    print(done)

[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]

[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]

[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]

[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]

[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]

...

[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]

[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]

[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]

[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]
[True, True]

KeyboardInterrupt: 

In [14]:
[False]*5

[False, False, False, False, False]

In [232]:
phase

(8, 7, 6, 5, 4)

In [233]:
ampA = intcode_computer(8, deepcopy(software))

In [234]:
ampA.step(0)

Amp waits for input


(1, False)

In [235]:
ampB = intcode_computer(7, deepcopy(software))

In [236]:
ampB.step(1)

Amp waits for input


(2, False)

In [237]:
ampC = intcode_computer(6, deepcopy(software))

In [238]:
ampC.step(2)

Amp waits for input


(4, False)

In [239]:
ampD = intcode_computer(5, deepcopy(software))

In [240]:
ampD.step(4)

Amp waits for input


(5, False)

In [241]:
ampE = intcode_computer(4, deepcopy(software))

In [242]:
ampE.step(4)

(100, True)